In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from datetime import datetime, timedelta

In [2]:
product_brand = pd.read_csv('./data/product_brand.csv', on_bad_lines='skip')
transaction = pd.read_csv('./data/transaction_new.csv')
customer = pd.read_csv('./data/customer.csv')

In [3]:
customer = customer.assign(first_join_date=pd.to_datetime(customer['first_join_date']))
transaction = transaction.assign(created_at=pd.to_datetime(transaction['created_at']))

In [4]:
customer = customer.assign(birthdate=pd.to_datetime(customer['birthdate']))
customer = customer.assign(age=2022-customer['birthdate'].dt.year)

In [5]:
transaction = transaction.assign(access_year = transaction['created_at'].dt.year)
transaction = transaction.assign(access_month = transaction['created_at'].dt.month)
transaction = transaction.assign(access_day = transaction['created_at'].dt.day)
transaction = transaction.assign(access_hour = transaction['created_at'].dt.hour)
transaction = transaction.assign(access_day_of_week = transaction['created_at'].dt.day_of_week)
transaction = transaction.assign(total_item_price = transaction['item_price'] * transaction['quantity'])
transaction

,created_at,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,...,total_amount,product_id,quantity,item_price,access_year,access_month,access_day,access_hour,access_day_of_week,total_item_price
0,2018-07-29 15:22:01.458193+00:00,5868,186e2bee-0637-4710-8981-50c2d737bc42,3abaa6ce-e320-4e51-9469-d9f3fa328e86,Debit Card,Success,1415,WEEKENDSERU,10000,2018-08-03T05:07:24.812676Z,...,199832,54728,1,191247,2018,7,29,15,6,191247
1,2018-07-30 12:40:22.365620+00:00,4774,caadb57b-e808-4f94-9e96-8a7d4c9898db,2ee5ead1-f13e-4759-92df-7ff48475e970,Credit Card,Success,0,NaN,10000,2018-08-03T01:29:03.415705Z,...,155526,16193,1,145526,2018,7,30,12,0,145526
2,2018-09-15 11:51:17.365620+00:00,4774,6000fffb-9c1a-4f4a-9296-bc8f6b622b50,93325fb6-eb00-4268-bb0e-6471795a0ad0,OVO,Success,0,NaN,10000,2018-09-18T08:41:49.422380Z,...,550696,53686,4,135174,2018,9,15,11,5,540696
3,2018-11-01 11:23:48.365620+00:00,4774,f5e530a7-4350-4cd1-a3bc-525b5037bcab,bcad5a61-1b67-448d-8ff4-781d67bc56e4,Credit Card,Success,0,NaN,0,2018-11-05T17:42:27.954235Z,...,271012,20228,1,271012,2018,11,1,11,3,271012
4,2018-12-18 11:20:30.365620+00:00,4774,0efc0594-dbbf-4f9a-b0b0-a488cfddf8a2,df1042ab-13e6-4072-b9d2-64a81974c51a,Credit Card,Success,0,NaN,0,2018-12-23T17:24:07.361785Z,...,198753,55220,1,198753,2018,12,18,11,1,198753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254580,2021-09-21 22:10:05.074873+00:00,25121,66f02fa4-c62e-497a-8b6e-be239009b50a,484eecde-9f4a-42de-a8df-7ce6668101f4,Debit Card,Failed,0,NaN,0,2021-09-27T00:43:40.992167Z,...,226820,30642,2,113410,2021,9,21,22,1,226820
1254581,2021-12-09 07:35:18.222046+00:00,34829,ccde6b83-ee42-4cce-b2cd-976574886eac,954e1211-4d45-4d0b-ba84-53e80119592c,OVO,Success,5010,AZ2022,10000,2021-12-12T07:52:44.803822Z,...,359203,19394,1,354213,2021,12,9,7,3,354213
1254582,2021-12-11 01:20:36.944593+00:00,99645,5bcd3434-1dc6-44bb-ae63-fd19714cbd6c,e6e6c108-abd0-49a2-b0a5-691117f9cfe1,Credit Card,Success,2181,WEEKENDSERU,5000,2021-12-12T13:28:22.848703Z,...,1158159,13079,4,288835,2021,12,11,1,5,1155340
1254583,2019-12-18 11:01:09.623382+00:00,87319,af7f291a-b01b-4bfd-89a9-237b16368b15,bb6e5426-a3f3-426a-b99d-79173eaa234b,Credit Card,Success,4503,AZ2022,0,2019-12-23T22:56:46.831720Z,...,184143,11312,1,188646,2019,12,18,11,2,188646


In [6]:
transaction_product = pd.merge(transaction, product_brand, how='left', left_on='product_id', right_on='id')
transaction_product_customer = pd.merge(transaction_product.rename(columns={'gender':'product_gender'}),
                                        customer[['customer_id', 'gender', 'first_join_date', 'age', 'home_location_lat', 'home_location_long', 'home_location']], how='left')
del product_brand
del transaction
del customer
transaction_product_customer

,created_at,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,...,year,usage,productDisplayName,brand,gender,first_join_date,age,home_location_lat,home_location_long,home_location
0,2018-07-29 15:22:01.458193+00:00,5868,186e2bee-0637-4710-8981-50c2d737bc42,3abaa6ce-e320-4e51-9469-d9f3fa328e86,Debit Card,Success,1415,WEEKENDSERU,10000,2018-08-03T05:07:24.812676Z,...,2012.0,Casual,vans men black shoes,vans,F,2018-07-28,22,-6.122897,106.876500,Jakarta Raya
1,2018-07-30 12:40:22.365620+00:00,4774,caadb57b-e808-4f94-9e96-8a7d4c9898db,2ee5ead1-f13e-4759-92df-7ff48475e970,Credit Card,Success,0,NaN,10000,2018-08-03T01:29:03.415705Z,...,2011.0,Casual,puma men knitted vest green sweater,puma,F,2018-07-28,26,-3.602334,120.363824,Sulawesi Selatan
2,2018-09-15 11:51:17.365620+00:00,4774,6000fffb-9c1a-4f4a-9296-bc8f6b622b50,93325fb6-eb00-4268-bb0e-6471795a0ad0,OVO,Success,0,NaN,10000,2018-09-18T08:41:49.422380Z,...,2012.0,Casual,kiara women purple & yellow handbag,kiara,F,2018-07-28,26,-3.602334,120.363824,Sulawesi Selatan
3,2018-11-01 11:23:48.365620+00:00,4774,f5e530a7-4350-4cd1-a3bc-525b5037bcab,bcad5a61-1b67-448d-8ff4-781d67bc56e4,Credit Card,Success,0,NaN,0,2018-11-05T17:42:27.954235Z,...,2011.0,Casual,wrangler women cable red sweater,wrangler,F,2018-07-28,26,-3.602334,120.363824,Sulawesi Selatan
4,2018-12-18 11:20:30.365620+00:00,4774,0efc0594-dbbf-4f9a-b0b0-a488cfddf8a2,df1042ab-13e6-4072-b9d2-64a81974c51a,Credit Card,Success,0,NaN,0,2018-12-23T17:24:07.361785Z,...,2017.0,Casual,lakme absolute forever silk chestnut lip liner 03,lakme,F,2018-07-28,26,-3.602334,120.363824,Sulawesi Selatan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254580,2021-09-21 22:10:05.074873+00:00,25121,66f02fa4-c62e-497a-8b6e-be239009b50a,484eecde-9f4a-42de-a8df-7ce6668101f4,Debit Card,Failed,0,NaN,0,2021-09-27T00:43:40.992167Z,...,2012.0,Casual,nike men printed white t-shirt,nike,M,2020-11-12,29,-6.869535,111.077145,Jawa Tengah
1254581,2021-12-09 07:35:18.222046+00:00,34829,ccde6b83-ee42-4cce-b2cd-976574886eac,954e1211-4d45-4d0b-ba84-53e80119592c,OVO,Success,5010,AZ2022,10000,2021-12-12T07:52:44.803822Z,...,2011.0,Casual,adidas men solid blue jacket,adidas,F,2021-12-08,16,-2.266407,120.415779,Sulawesi Selatan
1254582,2021-12-11 01:20:36.944593+00:00,99645,5bcd3434-1dc6-44bb-ae63-fd19714cbd6c,e6e6c108-abd0-49a2-b0a5-691117f9cfe1,Credit Card,Success,2181,WEEKENDSERU,5000,2021-12-12T13:28:22.848703Z,...,2011.0,Casual,numero uno men white casual shoes,numero uno,F,2021-12-08,31,-6.185498,106.831093,Jakarta Raya
1254583,2019-12-18 11:01:09.623382+00:00,87319,af7f291a-b01b-4bfd-89a9-237b16368b15,bb6e5426-a3f3-426a-b99d-79173eaa234b,Credit Card,Success,4503,AZ2022,0,2019-12-23T22:56:46.831720Z,...,2011.0,Casual,scullers men rugged chinos beige trousers,scullers,F,2019-12-18,15,-7.155977,110.891858,Jawa Tengah


In [7]:
paid_transaction_trend = transaction_product_customer[transaction_product_customer['payment_status']=='Success']
paid_transaction_trend

,created_at,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,...,year,usage,productDisplayName,brand,gender,first_join_date,age,home_location_lat,home_location_long,home_location
0,2018-07-29 15:22:01.458193+00:00,5868,186e2bee-0637-4710-8981-50c2d737bc42,3abaa6ce-e320-4e51-9469-d9f3fa328e86,Debit Card,Success,1415,WEEKENDSERU,10000,2018-08-03T05:07:24.812676Z,...,2012.0,Casual,vans men black shoes,vans,F,2018-07-28,22,-6.122897,106.876500,Jakarta Raya
1,2018-07-30 12:40:22.365620+00:00,4774,caadb57b-e808-4f94-9e96-8a7d4c9898db,2ee5ead1-f13e-4759-92df-7ff48475e970,Credit Card,Success,0,NaN,10000,2018-08-03T01:29:03.415705Z,...,2011.0,Casual,puma men knitted vest green sweater,puma,F,2018-07-28,26,-3.602334,120.363824,Sulawesi Selatan
2,2018-09-15 11:51:17.365620+00:00,4774,6000fffb-9c1a-4f4a-9296-bc8f6b622b50,93325fb6-eb00-4268-bb0e-6471795a0ad0,OVO,Success,0,NaN,10000,2018-09-18T08:41:49.422380Z,...,2012.0,Casual,kiara women purple & yellow handbag,kiara,F,2018-07-28,26,-3.602334,120.363824,Sulawesi Selatan
3,2018-11-01 11:23:48.365620+00:00,4774,f5e530a7-4350-4cd1-a3bc-525b5037bcab,bcad5a61-1b67-448d-8ff4-781d67bc56e4,Credit Card,Success,0,NaN,0,2018-11-05T17:42:27.954235Z,...,2011.0,Casual,wrangler women cable red sweater,wrangler,F,2018-07-28,26,-3.602334,120.363824,Sulawesi Selatan
4,2018-12-18 11:20:30.365620+00:00,4774,0efc0594-dbbf-4f9a-b0b0-a488cfddf8a2,df1042ab-13e6-4072-b9d2-64a81974c51a,Credit Card,Success,0,NaN,0,2018-12-23T17:24:07.361785Z,...,2017.0,Casual,lakme absolute forever silk chestnut lip liner 03,lakme,F,2018-07-28,26,-3.602334,120.363824,Sulawesi Selatan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254579,2020-11-12 18:19:20.074873+00:00,25121,7fdc05d1-234b-4889-a343-587ffc0635d5,33c3bf4f-0f24-4eb8-879d-906bf7e7736a,Debit Card,Success,0,NaN,10000,2020-11-18T18:06:06.493534Z,...,2012.0,Casual,french connection women grey belt,french connection,M,2020-11-12,29,-6.869535,111.077145,Jawa Tengah
1254581,2021-12-09 07:35:18.222046+00:00,34829,ccde6b83-ee42-4cce-b2cd-976574886eac,954e1211-4d45-4d0b-ba84-53e80119592c,OVO,Success,5010,AZ2022,10000,2021-12-12T07:52:44.803822Z,...,2011.0,Casual,adidas men solid blue jacket,adidas,F,2021-12-08,16,-2.266407,120.415779,Sulawesi Selatan
1254582,2021-12-11 01:20:36.944593+00:00,99645,5bcd3434-1dc6-44bb-ae63-fd19714cbd6c,e6e6c108-abd0-49a2-b0a5-691117f9cfe1,Credit Card,Success,2181,WEEKENDSERU,5000,2021-12-12T13:28:22.848703Z,...,2011.0,Casual,numero uno men white casual shoes,numero uno,F,2021-12-08,31,-6.185498,106.831093,Jakarta Raya
1254583,2019-12-18 11:01:09.623382+00:00,87319,af7f291a-b01b-4bfd-89a9-237b16368b15,bb6e5426-a3f3-426a-b99d-79173eaa234b,Credit Card,Success,4503,AZ2022,0,2019-12-23T22:56:46.831720Z,...,2011.0,Casual,scullers men rugged chinos beige trousers,scullers,F,2019-12-18,15,-7.155977,110.891858,Jawa Tengah


In [8]:
paid_transaction_groupby_session = paid_transaction_trend.groupby('session_id')[['customer_id', 'created_at', 'access_year', 'access_month', 'access_day', 'access_hour', 
                                                                                 'access_day_of_week', 'promo_code', 'total_amount', 'promo_amount', 'shipment_fee']].min()
paid_transaction_groupby_session = paid_transaction_groupby_session.assign(product_price = paid_transaction_groupby_session['total_amount']
                                                                           + paid_transaction_groupby_session['promo_amount']
                                                                           - paid_transaction_groupby_session['shipment_fee'])
paid_transaction_groupby_session = paid_transaction_groupby_session.assign(promotion_ratio = round(paid_transaction_groupby_session['promo_amount'] / 
                                                                                                   paid_transaction_groupby_session['product_price']*100, 2))
paid_transaction_groupby_session

,customer_id,created_at,access_year,access_month,access_day,access_hour,access_day_of_week,promo_code,total_amount,promo_amount,shipment_fee,product_price,promotion_ratio
session_id,,,,,,,,,,,,,
00001f29-3ede-4445-95f0-0178934e32ba,74433,2019-05-19 20:17:04.276396+00:00,2019,5,19,20,6,NaN,1427802,0,0,1427802,0.00
00003eca-954b-4150-aee1-63fc62f395cf,43805,2019-08-24 06:26:12.662344+00:00,2019,8,24,6,5,NaN,339790,0,0,339790,0.00
0000425b-ddc8-4459-a240-6ba814dac132,90074,2019-04-28 10:08:01.490443+00:00,2019,4,28,10,6,NaN,291698,0,10000,281698,0.00
00005746-17ec-449c-9c5c-699a3ea9304a,25255,2022-03-08 23:21:35.510057+00:00,2022,3,8,23,1,NaN,1231854,0,0,1231854,0.00
0000649b-9514-4600-979b-42166a798996,16868,2020-07-23 14:31:15.525079+00:00,2020,7,23,14,3,SC2022,70859,10288,10000,71147,14.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffffce53-1ba7-4d7d-a14a-ffa7079cfbff,544,2021-10-08 15:22:31.231938+00:00,2021,10,8,15,4,AZ2022,477264,3785,25000,456049,0.83
ffffd3d9-79db-459e-a427-388204a7aac7,15566,2021-11-28 03:26:04.983243+00:00,2021,11,28,3,6,LIBURDONG,4429048,2221,10000,4421269,0.05
ffffdf8e-dea1-42bc-9839-7662675c0d38,65771,2020-11-11 14:59:29.117552+00:00,2020,11,11,14,2,NaN,359771,0,10000,349771,0.00


# Promotion

In [12]:
week_day_promo = ['AZ2022', 'BUYMORE', 'XX2022', 'SC2022', 'STARTUP']
weekend_promo = ['WEEKENDMANTAP', 'WEEKENDSERU', 'LIBURDONG']

In [9]:
first_purchase_list = []

for _, group in tqdm(paid_transaction_groupby_session.groupby('customer_id')) : 
    temp = group.sort_values('created_at')
    first_purchase_list.append(temp.iloc[0].to_frame().T)

100%|██████████████████████████████████████████████████████████████████████████| 50242/50242 [00:47<00:00, 1059.75it/s]


In [10]:
first_purchase = pd.concat(first_purchase_list)
first_purchase

,customer_id,created_at,access_year,access_month,access_day,access_hour,access_day_of_week,promo_code,total_amount,promo_amount,shipment_fee,product_price,promotion_ratio
f03b6825-1e6e-4916-bf71-f94ecbcbaee5,3,2018-08-25 18:54:03.844494+00:00,2018,8,25,18,5,WEEKENDMANTAP,826597,8073,0,834670,0.97
cb2e3df3-6342-43e2-be2b-f19c79e92a1e,8,2021-02-13 09:47:39.656991+00:00,2021,2,13,9,5,WEEKENDSERU,223431,7699,0,231130,3.33
f23e77b3-d85c-4468-ada0-9f7fb9defe1e,9,2021-06-27 01:51:48.421034+00:00,2021,6,27,1,6,NaN,643165,0,0,643165,0.0
5f20fb20-8e35-4e30-a9d9-fc6a07d7835c,11,2022-03-07 14:29:13.759159+00:00,2022,3,7,14,0,XX2022,197533,9492,15000,192025,4.94
fe015013-09fd-4218-ab69-cb18706e63a7,15,2019-09-30 04:47:36.105040+00:00,2019,9,30,4,0,NaN,1078094,0,10000,1068094,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3dbdb18b-f498-48f6-aa86-8add876a97ec,99992,2020-06-30 21:03:37.873510+00:00,2020,6,30,21,1,NaN,275829,0,10000,265829,0.0
ed32902f-4a64-41ef-9626-981936ed4196,99995,2020-08-19 09:50:48.810263+00:00,2020,8,19,9,2,AZ2022,99627,5054,10000,94681,5.34
17e0c216-a441-4aff-986d-875fda0c1d8c,99997,2020-04-25 18:38:16.825127+00:00,2020,4,25,18,5,NaN,226461,0,10000,216461,0.0
d3047386-2968-4ec1-9b74-209a9223b542,99998,2018-07-02 20:56:19.703960+00:00,2018,7,2,20,0,AZ2022,211841,3181,15000,200022,1.59


In [17]:
(first_purchase.promo_code.count() / first_purchase.shape[0] *100).round(2)

35.85

## Seconde purchase promo

In [20]:
second_purchase_list = []

for _, group in tqdm(paid_transaction_groupby_session.groupby('customer_id')) : 
    temp = group.sort_values('created_at')
    try : 
        second_purchase_list.append(temp.iloc[1].to_frame().T)
    except : 
        continue

100%|██████████████████████████████████████████████████████████████████████████| 50242/50242 [00:39<00:00, 1272.52it/s]


In [21]:
second_purchase = pd.concat(second_purchase_list)
second_purchase

,customer_id,created_at,access_year,access_month,access_day,access_hour,access_day_of_week,promo_code,total_amount,promo_amount,shipment_fee,product_price,promotion_ratio
36673b2f-940a-42b5-994d-1cce4796b55f,3,2018-09-22 18:42:52.844494+00:00,2018,9,22,18,5,NaN,163784,0,10000,153784,0.0
5f093806-e7f4-495a-821e-1747f001a3d6,8,2021-04-30 12:01:35.656991+00:00,2021,4,30,12,4,NaN,328328,0,5000,323328,0.0
e4aa48ac-b265-4a1c-9d18-a8697443b3f3,9,2021-09-02 02:16:27.421034+00:00,2021,9,2,2,3,NaN,257065,0,10000,247065,0.0
358f77f5-faa7-49aa-adb1-101be57e4ffc,15,2020-04-24 02:53:09.105040+00:00,2020,4,24,2,4,BUYMORE,451937,5410,10000,447347,1.21
3816d1c4-29ad-4d7c-8bd1-23bfefdcb6a7,17,2020-09-27 13:52:19.521918+00:00,2020,9,27,13,6,NaN,129345,0,10000,119345,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7f817e96-3800-4b5f-acb5-aa56e70f5d9d,99989,2021-08-04 05:16:37.024729+00:00,2021,8,4,5,2,NaN,345901,0,10000,335901,0.0
ec9d33fe-8d23-443b-a688-0020171282bf,99991,2021-08-15 15:19:19.486520+00:00,2021,8,15,15,6,NaN,441789,0,15000,426789,0.0
8918023f-17f5-46ab-8b76-d9b1d47b8cd8,99992,2020-09-01 22:00:49.873510+00:00,2020,9,1,22,1,AZ2022,148610,4844,10000,143454,3.38
f7d541ed-7176-4a7b-b56a-432a56340e7a,99995,2020-08-31 10:29:02.810263+00:00,2020,8,31,10,0,NaN,93522,0,10000,83522,0.0


In [23]:
(second_purchase.promo_code.count() / second_purchase.shape[0] *100).round(2)

36.35